# Prediccion del precio de las viviendas mediante regresion lineal

En este script de realizara la prediccion mediantes metodos de regresion

pagina de guia (link)[https://joserzapata.github.io/courses/python-ciencia-datos/regresion/]

# Librerias

In [57]:
! pip install pandas -q

In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Datos

In [59]:
dfp=pd.read_csv(r'C:\Users\Juan Guzman\pviviendas\Data\dfanalisis')
dfp.head()

,tipo,precio,area,habitaciones,banos,garajes,balcon,trasn. publico,estrato,antiguedad,...,zonas verdes,supermercado o cc,instalacion de gas,parqueadero visitantes,colegios / universidades,parques cercanos,zona residencial,closet,municipio,tipo_propiedad
0,0,1380000000,320.00,5,6.0,2,1.0,1.0,5.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,rionegro,casa
1,1,496122600,91.97,3,3.0,1,1.0,1.0,3.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,rionegro,apartamento
2,0,570000000,150.00,3,3.0,1,1.0,1.0,5.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,san antonio de pereira,casa
3,0,290000000,106.00,3,3.0,1,0.0,0.0,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,marinilla,casa
4,1,360002000,73.37,3,3.0,0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,rionegro,apartamento


# Procesamiento de los datos


Primero dividiremos la variable objetivo que seria el precio de las variables explicativas que serian todas las demas

In [60]:
y = dfp['precio']
X_numeric = dfp.drop(['precio','municipio','tipo_propiedad'],axis=1)
X_categoric = dfp[['municipio','tipo_propiedad']]


## Procesamiento de variables numericas

para la regresión lineal múltiple, se recomienda que las variables predictoras y la variable objetivo tengan una distribución normal y que la relación entre ellas sea lineal. Si estas condiciones no se cumplen, se pueden aplicar transformaciones a las variables para mejorar la precisión del modelo.

Debido a que las distribuciones de las variables tienen comportamiento muy variados unas de otras se optara por estandarizar la variable lo que significa centrar  la variable en $0$ y con una desviación estándar de $1$:

$$X^{\prime} = \frac{X - \mu}{\sigma}$$


In [61]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() # declaramos el transformador
X_numeric_est = scaler.fit_transform(X_numeric)

## Procesamiento de variables categoricas

In [62]:
print(np.unique(X_categoric))

['apartaestudio' 'apartamento' 'casa' 'el carmen de viboral' 'el retiro'
 'finca' 'guarne' 'la ceja' 'llanogrande' 'marinilla' 'rionegro'
 'san antonio de pereira' 'santuario']


haremos el procesamiento mediante OneHotEncoder, el cual se asemeja al tratamiento de una variable dummi, es decir, para cada categoria se le asignara 1 si corresponde a esa categoria y 0 para las demas

In [63]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse=False)
X_categoric_onehot = enc.fit_transform(X_categoric)
X_categoric.shape , X_categoric_onehot.shape

((2144, 2), (2144, 13))

Podemos observar que se teniamos 13 categorias en las dos columnas

Ahora procederemos a unir los dos conjuntos de datos en la variable X

In [64]:
X = np.concatenate((X_numeric_est,X_categoric_onehot),axis=1)
dfp.shape , X.shape

((2144, 23), (2144, 33))

# Entrenamiento del modelo

Para este caso dividiremos los datos de la siguiente manera:
* 60% entrenamiento
* 20% test
* 20%validacion

Debido a que la regreison lineal no permite modificar muchos hiperparametros no entramos en la necesidad de utilizar un porcentaje de los datos para validar.
Con el fin de realizar un buen entrenamiento del algoritmo y mejorar su capacidad predictiva

In [65]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1234)
# particion de los datos de validacion en base a los datos de entrenamiento
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=1234)

In [66]:
from sklearn.model_selection import train_test_split

# Primera división en train y test (80% y 20% respectivamente)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Segunda división en train y validación (60% y 20% respectivamente)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=1234)

# Imprimir las dimensiones de cada conjunto
print(f"Conjunto de entrenamiento: {X_train.shape[0]}, {y_train.shape[0]}, en porcentaje correspone a {round(((X_train.shape[0]/X.shape[0])*100),0)} del total de la muestra")
print(f"Conjunto de validación: {X_val.shape[0]}, {y_val.shape[0]}, en porcentaje correspone a {round(((X_val.shape[0]/X.shape[0])*100),0)} del total de la muestra")
print(f"Conjunto de prueba: {X_test.shape[0]}, {y_test.shape[0]}, en porcentaje correspone a {round(((X_test.shape[0]/X.shape[0])*100),0)} del total de la muestra")


Conjunto de entrenamiento: 1286, 1286, en porcentaje correspone a 60.0 del total de la muestra
Conjunto de validación: 429, 429, en porcentaje correspone a 20.0 del total de la muestra
Conjunto de prueba: 429, 429, en porcentaje correspone a 20.0 del total de la muestra


# Modelo

## Regresion lineal multiple

Como se menciono anteriormente se utilizara una regresion lineal multiple para la predicccion

In [67]:
from sklearn.linear_model import LinearRegression

linear = LinearRegression()
linear.get_params()

{'copy_X': True,
 'fit_intercept': True,
 'n_jobs': None,
 'normalize': 'deprecated',
 'positive': False}

## Regresion Ridge

In [68]:
from sklearn.linear_model import Ridge
ridge = Ridge(random_state=1234)
ridge.get_params()

{'alpha': 1.0,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': None,
 'normalize': 'deprecated',
 'positive': False,
 'random_state': 1234,
 'solver': 'auto',
 'tol': 0.001}

## Regresion Lasso

In [69]:
from sklearn.linear_model import Lasso
lasso = Lasso(random_state=1234)
lasso.get_params()

{'alpha': 1.0,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': 1000,
 'normalize': 'deprecated',
 'positive': False,
 'precompute': False,
 'random_state': 1234,
 'selection': 'cyclic',
 'tol': 0.0001,
 'warm_start': False}

# Validacion cruzada

## Regresion lineal multiple

In [70]:
param_grid_linear = {'fit_intercept': [True, False],
                    'normalize': [True, False],
                    'copy_X': [True, False],
                    'positive': [True, False]
}
param_grid_linear

{'fit_intercept': [True, False],
 'normalize': [True, False],
 'copy_X': [True, False],
 'positive': [True, False]}

In [73]:
from sklearn.model_selection import GridSearchCV
gridlinear = GridSearchCV(linear, param_grid_linear, cv=5) # cv son las divisiones de la validacion cruzada

### Entrenamiento del modelo con GridSearchCV

In [ ]:
gridlinear.fit(X_train,y_train)

In [76]:
print(gridlinear.best_params_)
print(gridlinear.best_estimator_)

{'copy_X': True, 'fit_intercept': False, 'normalize': True, 'positive': False}
LinearRegression(fit_intercept=False, normalize=True)
